# Investigating Factors Affecting Housing Rental Yields (Using ONS Data)

## Purpose

The aim of this notebook is to use various public ONS (`https://www.ons.gov.uk/`) datasets to find factors affecting housing yields. I hope this will give readers (and potential investors) a better idea of what drives rental yields, hopefully allowing for better decision making.

Note: Here I look at yields instead of raw house prices since yields are comparable across different locations.

I approximate average yields using `median(rental income)/median(house price)` since the actual average yield, `median(rental income / house price)`, requires data for each house and is otherwise unavailable.


## Data Used

Below I list the ONS datasets I will be using. The full urls are given at the end.

- Crime
- Private Rental Statistics
- Median House Price
- Mean House Price
- Property Sales
- Population Estimates
- Quality of Life Estimates

These are the datasets I found on the ONS website I thought will be relevant and suitable to work with.

These give data reported by local authority so we will have a reasonable number of data points (roughly 300) to perform inference on.

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
# from ons_processing

In [1]:
!pip install -e .

Obtaining file:///home/jupyter
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ons-housing
    Found existing installation: ons-housing 0.0
    Can't uninstall 'ons-housing'. No files were found to uninstall.
  Running setup.py develop for ons-housing


In [10]:
from ons_processing.parse_files import *

In [46]:
ps = parse_property_sales(csv_dir)

/home/jupyter/ons_processing/parse_files.py:105: DtypeWarning: Columns (0,1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  df = load_df(csv_dir, "property_sales.csv")


In [49]:
ps.dropna(subset=["la_code"])

,la_code,ward_code,num_sold
0,E06000001,E05008945,113.0
1,E06000001,E05008946,173.0
2,E06000001,E05008947,70.0
3,E06000001,E05008943,56.0
4,E06000001,E05008944,100.0
...,...,...,...
8048,W06000024,W05000895,25.0
8049,W06000024,W05000707,38.0
8050,W06000024,W05000708,75.0
8051,W06000024,W05000710,16.0


In [44]:
cr = parse_crime(csv_dir)
cr

,la_code,num_households,total_crime,burgalry_per_household
1,E06000022,78700,55,6
2,E06000023,194200,104,11
3,E06000024,95100,62,5
4,Combined Local Authorities,245000,62,4
5,E06000025,117600,55,5
...,...,...,...,...
387,E08000034,180300,93,8
388,E08000035,331900,116,13
390,E08000036,150300,108,9
392,E06000030,94500,66,4


In [45]:
cr.la_code.isin(rs.la_code).value_counts()

True     273
False     38
Name: la_code, dtype: int64

In [30]:
rs = parse_rental_summary(csv_dir)
rs

,la_code,rent_count,median_rent,mean_rent
2,E06000047,5690,450,502
3,E06000005,2380,450,494
4,E06000001,860,475,485
5,E06000002,1260,475,490
6,E06000057,1910,480,536
...,...,...,...,...
351,E07000083,720,725,793
353,E07000187,1280,750,818
354,E07000188,1240,650,695
355,E07000246,1640,695,730


In [25]:
age = parse_population_age(csv_dir)
age

,la_code,la_name,ward_code,ward_name,total_population,child,adult,elderly
0,E09000002,Barking and Dagenham,E05000026,Abbey,16149,4905,10363,881
1,E09000002,Barking and Dagenham,E05000027,Alibon,10907,3301,6601,1005
2,E09000002,Barking and Dagenham,E05000028,Becontree,15182,4868,9056,1258
3,E09000002,Barking and Dagenham,E05000029,Chadwell Heath,11463,3478,6493,1492
4,E09000002,Barking and Dagenham,E05000030,Eastbrook,11557,2979,7016,1562
...,...,...,...,...,...,...,...,...
8048,W06000012,Neath Port Talbot,W05001034,Seven Sisters,2107,464,1169,474
8049,W06000012,Neath Port Talbot,W05001035,Tai-bach,4937,1054,2839,1044
8050,W06000005,Flintshire,W05001036,Broughton South,4220,961,2486,773
8051,W06000009,Pembrokeshire,W05001037,Penally,1784,288,871,625


In [17]:
wb = parse_wellbeing("/home/jupyter/ons_processing/data")
wb

,wellbeing_score,la_code,wellbeing_measure
21,7.12,E08000003,life-satisfaction
28,7.12,E08000003,happiness
29,7.41,E08000003,happiness
48,3.52,E08000003,anxiety
60,7.43,E08000003,worthwhile
...,...,...,...
84195,2.96,E07000035,anxiety
84196,8.13,E07000035,worthwhile
84197,8.02,E07000035,worthwhile
84198,3.04,E07000035,anxiety


In [12]:
csv_dir = "/home/jupyter/ons_processing/data"

## Data Sources

Crime
- https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea

Private Rental Statistics
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/privaterentalmarketsummarystatisticsinengland

Median House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/medianpricepaidbywardhpssadataset37

Mean House Price
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/meanpricepaidbywardhpssadataset38

Property Sales
- https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/numberofresidentialpropertysalesbywardhpssadataset36

Population Estimates
- https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/wardlevelmidyearpopulationestimatesexperimental

Quality of Life Estimates
- https://www.ons.gov.uk/datasets/wellbeing-local-authority/editions/time-series/versions/2

Note: the ONS has an API (https://developer.ons.gov.uk/) however these not all of these datasets are not available through this at the time of writing. Also, due issues parsing these excel files, I have manually exported the data to csvs which can be straightforwardly parsed.